<div class="alert alert-danger">
    <h4 style="font-weight: bold; font-size: 28px;">Feature Engineering</h4>
    <p style="font-size: 20px;">NBA API Data (2022-2024)</p>
</div>

<a name="Feature-Engineering"></a>

# Table of Contents

[Setup](#Setup)

[Data](#Data)

**[1. Create Team Matchups and Targets](#1.-Create-Team-Matchups-and-Targets)**

- [1.1. Clean Game Data](#1.1.-Clean-Game-Data)

- [1.2. Reshape to Game Matchups](#1.2.-Reshape-to-Game-Matchups)

- [1.3. Create Target Variables](#1.3.-Create-Target-Variables)

**[2. Create Rolling Window Statistics](#2.-Create-Rolling-Window-Statistics)**

# Setup

[Return to top](#Feature-Engineering)

In [1]:
import sys
from pathlib import Path
# get current working directory
cwd = %pwd
# add shared_code directory to Python sys.path
sys.path.append(str(Path(cwd).parent / "shared_code"))
# import all libraries in shared_code directory 'imports.py' file
from imports import *
%matplotlib inline

# Data

[Return to top](#Feature-Engineering)

In [8]:
misc_box_stats_df = pd.read_csv('../../data/original/nba_misc_boxscore_statistics_2021_2024.csv')
misc_box_stats_df.rename(columns={'minutes':'MIN'}, inplace=True)
misc_box_stats_df['MIN'] = misc_box_stats_df['MIN'].str.slice(0, 3)
misc_box_stats_df['MIN'] = misc_box_stats_df['MIN'].astype(int)
misc_box_stats_df.head()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,MIN,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,OPP_PTS_OFF_TOV,OPP_PTS_2ND_CHANCE,OPP_PTS_FB,OPP_PTS_PAINT,BLK,BLKA,PF,PFD,SEASON_ID,GAME_DATE,MATCHUP
0,22101221,1610612745,Houston Rockets,HOU,240,15.0,8.0,13.0,46.0,16.0,17.0,14.0,40.0,4,4,19,19,22021,2022-04-10,HOU vs. ATL
1,22101221,1610612737,Atlanta Hawks,ATL,240,16.0,17.0,14.0,40.0,15.0,8.0,13.0,46.0,4,4,19,19,22021,2022-04-10,ATL @ HOU
2,22101207,1610612748,Miami Heat,MIA,240,16.0,9.0,15.0,48.0,22.0,20.0,12.0,42.0,1,0,17,23,22021,2022-04-08,MIA vs. ATL
3,22101207,1610612737,Atlanta Hawks,ATL,240,22.0,20.0,12.0,42.0,16.0,9.0,15.0,48.0,0,1,23,17,22021,2022-04-08,ATL @ MIA
4,22101192,1610612764,Washington Wizards,WAS,240,8.0,11.0,13.0,44.0,16.0,10.0,11.0,40.0,4,1,17,13,22021,2022-04-06,WAS @ ATL


In [9]:
box_score_df = pd.read_csv('../../data/original/nba_games_box_scores_2022_2024.csv')

In [10]:
box_score_df.tail()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
7520,22023,1610612764,WAS,Washington Wizards,22300642,2024-01-27,WAS @ DET,W,240,118,45,100,0.450,11,34.0,0.324,17,21,0.810,16.0,34.0,50.0,26,10.0,4,9,19,14.0
7521,22023,1610612764,WAS,Washington Wizards,22300665,2024-01-29,WAS @ SAS,W,240,118,46,86,0.535,9,25.0,0.360,17,24,0.708,14.0,31.0,45.0,32,9.0,8,18,15,5.0
7522,22023,1610612764,WAS,Washington Wizards,22300676,2024-01-31,WAS vs. LAC,L,239,109,45,97,0.464,9,29.0,0.310,10,15,0.667,12.0,33.0,45.0,19,4.0,10,13,19,-16.0
7523,22023,1610612764,WAS,Washington Wizards,22300689,2024-02-02,WAS vs. MIA,L,239,102,37,90,0.411,11,42.0,0.262,17,21,0.810,6.0,37.0,43.0,28,5.0,4,8,25,-8.0
7524,22023,1610612764,WAS,Washington Wizards,22300705,2024-02-04,WAS vs. PHX,L,240,112,47,96,0.490,7,32.0,0.219,11,17,0.647,13.0,22.0,35.0,32,11.0,4,18,19,-28.0


In [11]:
misc_box_stats_df.head()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,MIN,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,OPP_PTS_OFF_TOV,OPP_PTS_2ND_CHANCE,OPP_PTS_FB,OPP_PTS_PAINT,BLK,BLKA,PF,PFD,SEASON_ID,GAME_DATE,MATCHUP
0,22101221,1610612745,Houston Rockets,HOU,240,15.0,8.0,13.0,46.0,16.0,17.0,14.0,40.0,4,4,19,19,22021,2022-04-10,HOU vs. ATL
1,22101221,1610612737,Atlanta Hawks,ATL,240,16.0,17.0,14.0,40.0,15.0,8.0,13.0,46.0,4,4,19,19,22021,2022-04-10,ATL @ HOU
2,22101207,1610612748,Miami Heat,MIA,240,16.0,9.0,15.0,48.0,22.0,20.0,12.0,42.0,1,0,17,23,22021,2022-04-08,MIA vs. ATL
3,22101207,1610612737,Atlanta Hawks,ATL,240,22.0,20.0,12.0,42.0,16.0,9.0,15.0,48.0,0,1,23,17,22021,2022-04-08,ATL @ MIA
4,22101192,1610612764,Washington Wizards,WAS,240,8.0,11.0,13.0,44.0,16.0,10.0,11.0,40.0,4,1,17,13,22021,2022-04-06,WAS @ ATL


<a name="1.-Create-Team-Matchups-and-Targets"></a>
# 1. Create Team Matchups and Targets

[Return to top](#Feature-Engineering)

<a name="1.1.-Clean-Game-Data"></a>
## 1.1. Clean Game Data

[Return to top](#Feature-Engineering)

We need to do three key things to clean the data:

1. Remove games with team aggregated game times of less than 238 minutes (which will remove exhibition matches).
2. Retain only games that are part of the regular season.
3. Remove any orphans (i.e., game IDs that do not have a partner) when reshaping to matchups.

Last 3 NBA regular seasons start and end dates:

- 2021-22 season: 2021-10-19 to 2022-04-10
- 2022-23 season: 2022-10-18 to 2023-04-09
- 2023-24 season: 2023-10-24 to 2024-04-14

In [12]:
# last 3 seasons start and end dates and labels
season_start_dates = ['2021-10-19', '2022-10-18', '2023-10-24']
season_end_dates   = ['2022-04-10', '2023-04-09', '2024-04-14']
season_labels      = ['2021-22', '2022-23', '2023-24']

In [13]:
# clean up the data
misc_box_stats_df_cleaned = utl.clean_team_bs_data(misc_box_stats_df, season_start_dates=season_start_dates, 
                                            season_end_dates=season_end_dates, season_labels=season_labels)

Season 2021-22: 1230 games
Season 2022-23: 1230 games
Season 2023-24: 842 games


In [14]:
# clean up the data
box_score_df_cleaned = utl.clean_team_bs_data(box_score_df, season_start_dates=season_start_dates, 
                                            season_end_dates=season_end_dates, season_labels=season_labels)

Season 2021-22: 1230 games
Season 2022-23: 1230 games
Season 2023-24: 736 games


In [15]:
#get GAME_DATE, MATCHUP, GAME_ID, TEAM_ABBREVIATION fields from games_df
misc_box_stats_df_cleaned = pd.merge(misc_box_stats_df_cleaned, box_score_df_cleaned[['WL','PTS','GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION', 'PLUS_MINUS']], on=['GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION'])

misc_box_stats_df_cleaned.sort_values(by=['PTS'])

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,MIN,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,OPP_PTS_OFF_TOV,OPP_PTS_2ND_CHANCE,OPP_PTS_FB,OPP_PTS_PAINT,BLK,BLKA,PF,PFD,SEASON_ID,GAME_DATE,MATCHUP,WL,PTS,PLUS_MINUS
890,22100075,1610612742,Dallas Mavericks,DAL,240,20.0,1.0,8.0,22.0,19.0,8.0,15.0,50.0,3,3,17,24,2021-22,2021-10-29,DAL @ DEN,L,75,-31.0
242,22100595,1610612752,New York Knicks,NYK,240,13.0,23.0,3.0,24.0,21.0,7.0,7.0,42.0,4,8,15,15,2021-22,2022-01-08,NYK @ BOS,L,75,-24.0
224,22100717,1610612758,Sacramento Kings,SAC,240,11.0,10.0,14.0,34.0,21.0,28.0,12.0,56.0,8,6,14,12,2021-22,2022-01-25,SAC @ BOS,L,75,-53.0
6222,22300529,1610612757,Portland Trail Blazers,POR,240,12.0,16.0,11.0,30.0,12.0,17.0,14.0,70.0,8,8,19,10,2023-24,2024-01-11,POR @ OKC,L,77,-62.0
727,22100257,1610612741,Chicago Bulls,CHI,240,5.0,7.0,7.0,32.0,16.0,11.0,19.0,46.0,2,12,16,15,2021-22,2021-11-22,CHI vs. IND,L,77,-32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3481,22201230,1610612744,Golden State Warriors,GSW,240,25.0,9.0,26.0,60.0,17.0,14.0,14.0,48.0,6,3,18,9,2022-23,2023-04-09,GSW @ POR,W,157,56.0
4975,22300039,1610612754,Indiana Pacers,IND,240,19.0,12.0,20.0,64.0,17.0,16.0,17.0,74.0,11,4,25,23,2023-24,2023-11-21,IND @ ATL,W,157,5.0
2049,22100723,1610612766,Charlotte Hornets,CHA,240,30.0,13.0,30.0,54.0,18.0,19.0,15.0,60.0,5,3,29,31,2021-22,2022-01-26,CHA @ IND,W,158,32.0
3751,22200902,1610612746,LA Clippers,LAC,290,27.0,7.0,17.0,48.0,42.0,12.0,41.0,88.0,3,3,27,28,2022-23,2023-02-24,LAC vs. SAC,L,175,-1.0


<a name="1.2.-Reshape-to-Game-Matchups"></a>
## 1.2. Reshape to Game Matchups

[Return to top](#Feature-Engineering)

In [16]:
# identify non-stats columns
non_stats_cols = ['SEASON_ID', 'GAME_ID', 'GAME_DATE', 'MATCHUP']

# reshape team box score data to wide format so each row is a game matchup
misc_box_stats_matchups_df = utl.reshape_team_bs_to_matchups(misc_box_stats_df_cleaned, non_stats_cols)

Season 2021-22: 1222 games
Season 2022-23: 1221 games
Season 2023-24: 728 games


In [17]:
misc_box_stats_matchups_df.head()

,GAME_ID,HOME_TEAM_ID,HOME_TEAM_NAME,HOME_TEAM_ABBREVIATION,HOME_MIN,HOME_PTS_OFF_TOV,HOME_PTS_2ND_CHANCE,HOME_PTS_FB,HOME_PTS_PAINT,HOME_OPP_PTS_OFF_TOV,HOME_OPP_PTS_2ND_CHANCE,HOME_OPP_PTS_FB,HOME_OPP_PTS_PAINT,HOME_BLK,HOME_BLKA,HOME_PF,HOME_PFD,SEASON_ID,GAME_DATE,HOME_WL,HOME_PTS,HOME_PLUS_MINUS,AWAY_TEAM_ID,AWAY_TEAM_NAME,AWAY_TEAM_ABBREVIATION,AWAY_MIN,AWAY_PTS_OFF_TOV,AWAY_PTS_2ND_CHANCE,AWAY_PTS_FB,AWAY_PTS_PAINT,AWAY_OPP_PTS_OFF_TOV,AWAY_OPP_PTS_2ND_CHANCE,AWAY_OPP_PTS_FB,AWAY_OPP_PTS_PAINT,AWAY_BLK,AWAY_BLKA,AWAY_PF,AWAY_PFD,AWAY_WL,AWAY_PTS,AWAY_PLUS_MINUS
0,22101221,1610612745,Houston Rockets,HOU,240,15.0,8.0,13.0,46.0,16.0,17.0,14.0,40.0,4,4,19,19,2021-22,2022-04-10,L,114,-16.0,1610612737,Atlanta Hawks,ATL,240,16.0,17.0,14.0,40.0,15.0,8.0,13.0,46.0,4,4,19,19,W,130,16.0
1,22101207,1610612748,Miami Heat,MIA,240,16.0,9.0,15.0,48.0,22.0,20.0,12.0,42.0,1,0,17,23,2021-22,2022-04-08,W,113,4.0,1610612737,Atlanta Hawks,ATL,240,22.0,20.0,12.0,42.0,16.0,9.0,15.0,48.0,0,1,23,17,L,109,-4.0
2,22101192,1610612737,Atlanta Hawks,ATL,240,16.0,10.0,11.0,40.0,8.0,11.0,13.0,44.0,1,4,13,17,2021-22,2022-04-06,W,118,15.0,1610612764,Washington Wizards,WAS,240,8.0,11.0,13.0,44.0,16.0,10.0,11.0,40.0,4,1,17,13,L,103,-15.0
3,22101182,1610612761,Toronto Raptors,TOR,240,9.0,24.0,11.0,62.0,9.0,16.0,11.0,42.0,5,4,14,22,2021-22,2022-04-05,W,118,10.0,1610612737,Atlanta Hawks,ATL,240,9.0,16.0,11.0,42.0,9.0,24.0,11.0,62.0,4,5,22,14,L,108,-10.0
4,22101163,1610612737,Atlanta Hawks,ATL,240,15.0,14.0,12.0,36.0,12.0,8.0,15.0,36.0,4,3,18,29,2021-22,2022-04-02,W,122,7.0,1610612751,Brooklyn Nets,BKN,240,12.0,8.0,15.0,36.0,15.0,14.0,12.0,36.0,3,4,29,18,L,115,-7.0


<a name="1.3.-Create-Target-Variables"></a>
## 1.3. Create Target Variables

[Return to top](#Feature-Engineering)

There are three targets of interest:

1. **Total Game Points (over / under):** This can be calculated as the sum of `HOME_PTS + AWAY_PTS`.
2. **Difference in Game Points (plus / minus):** This can be calculated in relation to the home team as the following difference: `HOME_PTS - AWAY_PTS`.
3. **Game Winner (moneyline):** This can be defined in relation to the home team using the `HOME_WL` column, where a win for the home team is equal to 1 and a loss for the home team equal to 0. We will create a new column called `GAME_RESULT` for this indicator.

In [18]:
# create the above three target variables
misc_box_stats_matchups_df = utl.create_target_variables(misc_box_stats_matchups_df, 'HOME_WL', 'HOME_PTS', 'AWAY_PTS')

In [19]:
misc_box_stats_matchups_df[['GAME_DATE', 'GAME_ID',  'HOME_TEAM_NAME', 'AWAY_TEAM_NAME', 'HOME_PTS', 'AWAY_PTS', 'GAME_RESULT', 'TOTAL_PTS', 'PLUS_MINUS']].tail()

,GAME_DATE,GAME_ID,HOME_TEAM_NAME,AWAY_TEAM_NAME,HOME_PTS,AWAY_PTS,GAME_RESULT,TOTAL_PTS,PLUS_MINUS
3166,2023-11-22,22300225,Charlotte Hornets,Washington Wizards,117,114,1,231,3.0
3167,2023-11-10,22300009,Washington Wizards,Charlotte Hornets,117,124,0,241,-7.0
3168,2023-11-08,22300157,Charlotte Hornets,Washington Wizards,116,132,0,248,-16.0
3169,2024-01-24,22300619,Detroit Pistons,Charlotte Hornets,113,106,1,219,7.0
3170,2023-10-27,22300077,Charlotte Hornets,Detroit Pistons,99,111,0,210,-12.0


<a name="2.-Create-Rolling-Window-Statistics"></a>
# 2. Create Rolling Window Statistics

[Return to top](#Feature-Engineering)

Here we create average box scores for each team over a rolling window of the previous $n$-games.

In [20]:
# identify stats columns
non_stats_cols = ['SEASON_ID', 'GAME_ID', 'GAME_DATE', 'HOME_TEAM_ID', 'AWAY_TEAM_ID',
                  'HOME_TEAM_NAME', 'AWAY_TEAM_NAME', 'HOME_WL', 'AWAY_WL', 'HOME_MIN', 
                  'AWAY_MIN', 'HOME_TEAM_ABBREVIATION', 'AWAY_TEAM_ABBREVIATION']
stats_cols = [col for col in misc_box_stats_matchups_df.columns if col not in non_stats_cols]

In [27]:
# calculate rolling averages for each statistic and add them to the DataFrame
misc_box_stats_matchups_roll_df = utl.process_rolling_stats(
    misc_box_stats_matchups_df, 
    stats_cols, 
    target_cols=['GAME_RESULT', 'TOTAL_PTS', 'PLUS_MINUS'],
    window_size=5,   # the number of games to include in the rolling window
    min_obs=1,       # the minimum number of observations present within the window to yield an aggregate value
    stratify_by_season=True,  # should the rolling calculations be reset at the start of each new season or be contiguous across seasons? 
    exclude_initial_games=0   # number of initial games to exclude from the rolling averages (optionally by season)
)

In [28]:
misc_box_stats_matchups_roll_df.tail()

,GAME_ID,GAME_RESULT,TOTAL_PTS,PLUS_MINUS,HOME_TEAM_NAME,SEASON_ID,GAME_DATE,ROLL_HOME_PTS_OFF_TOV,ROLL_HOME_PTS_2ND_CHANCE,ROLL_HOME_PTS_FB,ROLL_HOME_PTS_PAINT,ROLL_HOME_OPP_PTS_OFF_TOV,ROLL_HOME_OPP_PTS_2ND_CHANCE,ROLL_HOME_OPP_PTS_FB,ROLL_HOME_OPP_PTS_PAINT,ROLL_HOME_BLK,ROLL_HOME_BLKA,ROLL_HOME_PF,ROLL_HOME_PFD,ROLL_HOME_PTS,AWAY_TEAM_NAME,ROLL_AWAY_PTS_OFF_TOV,ROLL_AWAY_PTS_2ND_CHANCE,ROLL_AWAY_PTS_FB,ROLL_AWAY_PTS_PAINT,ROLL_AWAY_OPP_PTS_OFF_TOV,ROLL_AWAY_OPP_PTS_2ND_CHANCE,ROLL_AWAY_OPP_PTS_FB,ROLL_AWAY_OPP_PTS_PAINT,ROLL_AWAY_BLK,ROLL_AWAY_BLKA,ROLL_AWAY_PF,ROLL_AWAY_PFD,ROLL_AWAY_PTS
2540,22300703,0,218,-16.0,San Antonio Spurs,2023-24,2024-02-03,15.85,12.20,16.55,49.5,17.30,13.75,14.65,55.1,6.75,4.50,18.20,18.50,114.60,Cleveland Cavaliers,16.50,13.40,14.05,49.3,17.75,12.95,13.65,45.7,4.25,5.30,20.1,20.20,112.80
3077,22300705,0,252,-28.0,Washington Wizards,2023-24,2024-02-04,15.30,10.40,16.30,56.1,17.25,18.80,14.45,59.8,5.60,5.25,19.80,17.30,115.10,Phoenix Suns,16.50,13.40,14.90,48.2,18.05,15.75,13.30,50.0,6.45,4.25,18.1,20.10,119.85
3017,22300704,0,210,-12.0,Detroit Pistons,2023-24,2024-02-04,12.30,12.05,14.00,51.3,17.30,13.40,14.80,57.9,4.85,5.70,21.00,18.40,113.05,Orlando Magic,18.10,15.05,14.40,51.6,15.95,14.15,14.35,51.8,5.25,5.05,21.3,21.60,111.70
3036,22300707,0,214,-18.6,Charlotte Hornets,2023-24,2024-02-04,14.75,12.50,12.30,47.4,17.20,13.50,11.60,56.6,4.90,5.75,18.85,17.75,107.55,Indiana Pacers,17.65,16.20,16.30,56.8,17.35,14.15,13.25,60.7,6.40,5.55,23.3,19.25,121.90
2492,22300706,1,222,40.0,Boston Celtics,2023-24,2024-02-04,16.60,14.55,15.00,44.2,14.00,15.65,11.65,46.3,6.55,3.55,17.85,17.95,121.05,Memphis Grizzlies,17.55,11.05,13.10,47.4,17.55,13.85,14.05,51.3,6.45,6.05,20.1,20.10,110.10


In [29]:
# write out the matchups with rolling features
misc_box_stats_matchups_roll_df.to_csv('../../data/processed/nba_team_matchups_rolling_misc_box_stats_2021_2024_r05.csv', index=False)